In [1]:
import importlib
import sys
from urllib.request import urlretrieve

# Imports.
import numpy as np
import meshcat
from ipywidgets import interact, FloatSlider, ToggleButton
from IPython.display import display

import pydrake.all
from pydrake.all import (AddMultibodyPlantSceneGraph, DiagramBuilder, PlanarSceneGraphVisualizer, SceneGraph, Simulator)
from pydrake.systems.jupyter_widgets import WidgetSystem
from pydrake.examples.pendulum import PendulumGeometry, PendulumPlant

import underactuated
from underactuated.jupyter import AdvanceToAndVisualize, SetupMatplotlibBackend, running_as_notebook
import underactuated.meshcat_utils as mutil
from pydrake.examples.pendulum import PendulumGeometry, PendulumPlant, PendulumInput
from pydrake.all import Linearize, LinearQuadraticRegulator,VectorSystem
import math
from pydrake.all import Jacobian, MathematicalProgram, Solve, Variables, Polynomial
import numpy as np

In [2]:
# LQR
m = 1
M = 5
L = 2
g = -10
d = 1
b = 1

A = np.array([[0,1,0,0], [0,-d/M,b*m*g/M,0], [0,0,0,1], [0,-b*d/(M*L),-b*(m+M)*g/(M*L),0]])
B = np.array([[0],[1/M], [0], [b*1/(M*L)]])

Q = np.diag((10., 1. , 10., 1.))
R = [1.]
(K, S) = LinearQuadraticRegulator(A, B, Q, R)
print(S)

[[  25.08993933   30.47525278 -244.62531583  -92.57328217]
 [  30.47525278   54.70327322 -521.19015116 -198.74790108]
 [-244.62531583 -521.19015116 7654.55505719 2991.57725726]
 [ -92.57328217 -198.74790108 2991.57725726 1171.06897352]]


In [3]:
### Prepare non-linear system

# non-linear dyamics
prog = MathematicalProgram()
x = prog.NewIndeterminates(4, "x")
x1 = x[0]
x2 = x[1]
x3 = x[2]
x4 = x[3]
pi = np.pi
r = [0, 0, pi, 0]  # reference
f1 = x2
f2 = (2233260595080309*x2)/1407374883553280 - (3253141686419969*x3)/87960930222080 - (1360884314958167*x4)/87960930222080 + (3253141686419969*pi)/87960930222080 + (10**(1/2)*x1)/5 - (2233260595080309*x2*(x3 - pi)**2)/7036874417766400 + (1360884314958167*x4*(x3 - pi)**2)/439804651110400 - (2*x4**2*(x3 - pi))/5 + (8000206454818307*(x3 - pi)**3)/1319413953331200 - (10**(1/2)*x1*(x3 - pi)**2)/25
f3 = x4
f4 = (2233260595080309*x2)/2814749767106560 - (2373532384199169*x3)/175921860444160 - (1360884314958167*x4)/175921860444160 + (2373532384199169*pi)/175921860444160 + (10**(1/2)*x1)/10 - (15632824165562163*x2*(x3 - pi)**2)/28147497671065600 + (9526190204707169*x4*(x3 - pi)**2)/1759218604441600 - (x4**2*(x3 - pi))/5 + (20133163898277383*(x3 - pi)**3)/1759218604441600 - (7*10**(1/2)*x1*(x3 - pi)**2)/100
 
fn = [f1,f2,f3,f4]

In [4]:
### Lyapunov analysis, fix V and find Lagrange multiplier.

# cost-to-go of LQR as Lyapunov candidate
V = (x-r).dot(S.dot(x-r)) 
Vdot = Jacobian([V], x).dot(fn)[0]

# Define the Lagrange multiplier.
lambda_ = prog.NewSosPolynomial(Variables(x), 4)[0].ToExpression()

# Optimization setup
rho = 16
prog.AddSosConstraint(-Vdot + lambda_*(V-rho))
eps = 1e-4
prog.AddSosConstraint(V - eps * (x - r).dot(x - r))
prog.AddSosConstraint(lambda_)

result = Solve(prog)

# Print result
if (result.is_success()):
    print(f"Verified that {str(V)} < {rho} is in the region of attraction.")
    #print(Polynomial(result.GetSolution(lambda_)))
else:
    print("failed")

Verified that ((x(0) * (25.089939332035222 * x(0) + 30.475252784260839 * x(1) - 92.573282170195171 * x(3) - 244.62531583054917 * (-3.1415926535897931 + x(2)))) + (x(1) * (30.475252784261009 * x(0) + 54.703273217802973 * x(1) - 198.74790108026389 * x(3) - 521.19015115668242 * (-3.1415926535897931 + x(2)))) + (x(3) * ( - 92.573282170197402 * x(0) - 198.74790108026656 * x(1) + 1171.0689735231294 * x(3) + 2991.5772572603487 * (-3.1415926535897931 + x(2)))) + ((-3.1415926535897931 + x(2)) * ( - 244.62531583055468 * x(0) - 521.1901511566889 * x(1) + 2991.5772572603469 * x(3) + 7654.5550571892891 * (-3.1415926535897931 + x(2))))) < 16 is in the region of attraction.


In [14]:
x1 = 0
x2 = 0
x3 = pi+0.01
x4 = 0.04
V =((x1* (25.089939332035222 * x1+ 30.475252784260839 * x2- 92.573282170195171 * x4- 244.62531583054917 * (-3.1415926535897931 + x3))) + (x2* (30.475252784261009 * x1+ 54.703273217802973 * x2- 198.74790108026389 * x4- 521.19015115668242 * (-3.1415926535897931 + x3))) + (x4* ( - 92.573282170197402 * x1- 198.74790108026656 * x2+ 1171.0689735231294 * x4+ 2991.5772572603487 * (-3.1415926535897931 + x3))) + ((-3.1415926535897931 + x3) * ( - 244.62531583055468 * x1- 521.1901511566889 * x2+ 2991.5772572603469 * x4+ 7654.5550571892891 * (-3.1415926535897931 + x3))))
print(V)

5.0324276691641305
